In [8]:
import numpy as np
import urllib
import os
import glob
import pathlib
import zipfile
import geojson
import geopandas
from geojson import Polygon as geoPolygon
from geojson import Point as geoPoint
from geojson import Polygon,FeatureCollection,dump
from pathlib import Path
import pandas as pd
import geopandas as gpd
from geojson_utils import centroid

In [9]:
def create_geoJson(folderPath,biome):
# Finding txt meta file
    str_path = str(folderPath)
    print(str(folderPath))
    meta_file = glob.glob(str_path + '/*.txt')
    try:
        meta_file = str(meta_file[0])
        basefilename = pathlib.PurePath(folderPath).name
        print(basefilename)
        # Getting meta data
        meta = GetLandsatMetadata(meta_file)  # create object
        product_metadata = meta.get('L1_METADATA_FILE').get('PRODUCT_METADATA')
        ## First
        lat_long_features = geoPolygon([[(product_metadata.get('CORNER_UL_LON_PRODUCT'),
                                   product_metadata.get('CORNER_UL_LAT_PRODUCT')), (
                                  product_metadata.get('CORNER_LL_LON_PRODUCT'),
                                  product_metadata.get('CORNER_LL_LAT_PRODUCT')),
                                  (product_metadata.get('CORNER_LR_LON_PRODUCT'),
                                   product_metadata.get('CORNER_LR_LAT_PRODUCT')), (
                                  product_metadata.get('CORNER_UR_LON_PRODUCT'),
                                  product_metadata.get('CORNER_UR_LAT_PRODUCT')), (
                                  product_metadata.get('CORNER_UL_LON_PRODUCT'),
                                  product_metadata.get('CORNER_UL_LAT_PRODUCT'))]])

        basefilename = os.path.join(folderPath, basefilename)
        cur_centroid = centroid(lat_long_features)
        cur_centroid = cur_centroid.get('coordinates')
        lat_long_features = geoPoint((cur_centroid[0],cur_centroid[1]))
        geom_in_geojson = []
        geom_in_geojson.append(geojson.Feature(geometry=lat_long_features, properties={"Biome":str(biome),
            "Data_Acquired":product_metadata.get('DATE_ACQUIRED'),
            "ProductType":product_metadata.get('SPACECRAFT_ID'),
            "centroid_lon":cur_centroid[0],
            "centroid_lat": cur_centroid[1]}))
        feature_collection = FeatureCollection(geom_in_geojson)
        pathToFile = str(basefilename) + '.geojson'
        with open(pathToFile, 'w') as f:
            dump(feature_collection, f)
    except OSError:
        pass
    
    
    

In [16]:
def removeFiles(path):
    for root, dirs, files in os.walk(path):
        for currentFile in files:
            exts = ('.tif', '.hdr','img','pkl')
            if currentFile.lower().endswith(exts):
                os.remove(os.path.join(root, currentFile))

In [11]:
    def GetLandsatMetadata(filename):
        s = ReadFile(filename)
        (items, junk) = Parse(s)
        return items

    def ReadFile(filename):
        f = open(filename, "r", encoding="utf-8")
        s = f.read()
        f.close()
        return s

    def ParseValue(str):
        try:
            try:
                return int(str)
            except ValueError:
                return float(str)
        except ValueError:
            if str[0] == str[-1] == '"':
                str = str[1:-1]
            return str

    def GetLine(s, newline='\n'):
        next = s.find(newline, 0, None)
        line = s[0:next]
        s = s[next + 1:]
        return (line, s)

    def Parse(contents, newline='\n'):
        result = {}
        separator = ' = '
        while len(contents) > 0:
            (line, contents) = GetLine(contents)
            line = line.strip()
            if line == 'END':
                return (result, contents)
            if len(line) == 0:
                continue
            parts = line.split(separator)
            if len(parts) < 2:
                continue
            if parts[0] == 'GROUP':
                subgroup = parts[1]
                (result[subgroup], contents) = Parse(contents, newline)
            elif parts[0] == 'END_GROUP':
                return (result, contents)
            else:
                name = parts[0]
                value = line[len(name) + len(separator):].strip()
                value = ParseValue(value)
                result[name] = value
        return (result, contents)

    def write_gjson(features,name):
        # feature is a shapely geometry type
        geom_in_geojson = []
        geom_in_geojson.append(geojson.Feature(geometry=features, properties={}))
        feature_collection = FeatureCollection(geom_in_geojson)
        with open(name+'Lat_Long.geojson', 'w') as f:
            dump(feature_collection, f)
        return Path(name+'.geojson')

In [5]:
# Get the webpage
url = 'https://landsat.usgs.gov/landsat-8-cloud-cover-assessment-validation-data'
response = urllib.request.urlopen(url)

# Find the data urls in the source
data_urls = []
for line in response:
    if 'https://landsat.usgs.gov/cloud-validation/cca' in str(line) and 'tar' in str(line):        
        data_urls.append(str(line)[47:124])

print("Products found:", np.size(data_urls))
print("Example url:   ", data_urls[0])

Products found: 96
Example url:    https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz


In [ ]:
# Download all the products
backgrounds = ['Barren',
               'Forest',
               'GrassCrops',
               'Shrubland',
               'SnowIce',
               'Urban',
               'Water',
               'Wetlands']
index = -1
for i in range(0, np.size(data_urls)):
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1     
    !wget {data_urls[i]} -P {'./tar/' + backgrounds[index] + '/'}

--2021-06-07 12:22:24--  https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz
Resolving landsat.usgs.gov (landsat.usgs.gov)... 152.61.136.10, 2001:49c8:4000:122c::10
Connecting to landsat.usgs.gov (landsat.usgs.gov)|152.61.136.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914473396 (872M) [application/x-gzip]
Saving to: ‘./tar/Barren/LC80420082013220LGN00.tar.gz.1’

LC80420082013220LGN 100%[===================>] 872.11M   979KB/s    in 21m 29s 

2021-06-07 12:43:53 (693 KB/s) - ‘./tar/Barren/LC80420082013220LGN00.tar.gz.1’ saved [914473396/914473396]

--2021-06-07 12:43:54--  https://landsat.usgs.gov/cloud-validation/cca_l8/LC80500092014231LGN00.tar.gz
Resolving landsat.usgs.gov (landsat.usgs.gov)... 152.61.136.10, 2001:49c8:4000:122c::10
Connecting to landsat.usgs.gov (landsat.usgs.gov)|152.61.136.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1014416376 (967M) [application/x-gzip]
Saving to: ‘./tar/

In [17]:
# Unzip all files into their respective folders
folders = sorted(os.listdir("tar"))
home_path = os.path.join(os.getcwd(),'unzipped')
try:
    os.mkdir(home_path)
except OSError:
    pass
for folder in folders: 
    home_path = os.path.join(os.getcwd(),'unzipped')
    home_path = os.path.join(home_path,folder)
    print(str(home_path))
    if not os.path.exists(home_path):
        os.makedirs(home_path)
    files = sorted(os.listdir("tar/" + folder))
    files = [f for f in files if '.tar.gz' in f] 
    for file in files:
            print("----")
            print("Unzipping product:", file)
            #!tar -xvzf {"./tar/" + folder + '/' + file} -C {'./unzipped/' + folder + '/'}
            file_name=file.split('.')[0]
            nested_folder = sorted(os.listdir('./unzipped/' + folder + '/'))
            unzipped_fullpath=os.path.join(home_path,nested_folder[0])
            #unzipped_fullpath=os.path.join(home_path,'BC')
            current_folder=os.path.join(unzipped_fullpath,file_name)
            ##create_geoJson(current_folder,folder)
            ## Removing tif files
            removeFiles(current_folder)
            

/workspace/data_landset8/unzipped/Barren
----
Unzipping product: LC80420082013220LGN00.tar.gz
----
Unzipping product: LC80420082013220LGN00.tar.gz.1
----
Unzipping product: LC80500092014231LGN00.tar.gz
----
Unzipping product: LC80500092014231LGN00.tar.gz.1
----
Unzipping product: LC80530022014156LGN00.tar.gz
----
Unzipping product: LC80530022014156LGN00.tar.gz.1
----
Unzipping product: LC81330312013202LGN00.tar.gz
----
Unzipping product: LC81330312013202LGN00.tar.gz.1
----
Unzipping product: LC81360302014162LGN00.tar.gz
----
Unzipping product: LC81360302014162LGN00.tar.gz.1
----
Unzipping product: LC81390292014135LGN00.tar.gz
----
Unzipping product: LC81390292014135LGN00.tar.gz.1
----
Unzipping product: LC81550082014263LGN00.tar.gz
----
Unzipping product: LC81550082014263LGN00.tar.gz.1
----
Unzipping product: LC81570452014213LGN00.tar.gz
----
Unzipping product: LC81570452014213LGN00.tar.gz.1
----
Unzipping product: LC81640502013179LGN01.tar.gz
----
Unzipping product: LC81640502013179LG